In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install --upgrade torch==2.0.0 torchvision

In [ ]:
# !pip install apache_beam mwparserfromhell
# !pip install datasets

In [ ]:
## public libraries
from datasets import load_dataset
import torch

## local libraries
from gpt_utils import (
    get_encoder_decoder_size,
    text_to_tv_tensors,
    get_batch,
    training_run,
    test_forward_pass,
    test_gen_text
)
from attention_decoder import AttentionModule


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.set_default_device(device)
device

In [ ]:
wiki_raw = load_dataset("wikipedia", "20220301.simple")

In [ ]:
wiki_raw

In [ ]:
wiki_text_blob = '\n\n'.join(wiki_raw['train']['text'])
print(len(wiki_text_blob))
print()
print(wiki_text_blob[:1000])

In [ ]:
##When sorted, low-numbered codes seem to be more "normal"
normal_chars = ''.join(sorted(list(set(wiki_text_blob)))[0:97])
normal_chars

In [ ]:
# 30 sec ish
wiki_text_blob_clean = ''.join([x for x in wiki_text_blob if x in normal_chars])

In [ ]:
# Keep 99.7% of chars. great.
len(wiki_text_blob_clean)/len(wiki_text_blob)

In [ ]:
encode, decode, vocab_size = get_encoder_decoder_size(wiki_text_blob_clean)
print(vocab_size)

In [ ]:
train, validate = text_to_tv_tensors(wiki_text_blob_clean, encode, device)

In [ ]:
m_attn = AttentionModule(vocab_size).to(device)

In [ ]:
test_forward_pass(m_attn, validate)

In [ ]:
# CPU = ~25s per epoch
training_run(m_attn, train, validate, num_epochs=5)

In [ ]:
test_gen_text(m_attn, "To be or not to ", encode, decode, device)
